In [1]:
! nvidia-smi

Sat May 28 07:49:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install pytorch-tabnet optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 31.5 MB/s 
     |████████████████████████████████| 81 kB 10.9 MB/s 
     |████████████████████████████████| 210 kB 60.9 MB/s 
     |████████████████████████████████| 78 kB 7.5 MB/s 
     |████████████████████████████████| 146 kB 73.6 MB/s 
     |████████████████████████████████| 112 kB 68.1 MB/s 
     |████████████████████████████████| 49 kB 4.9 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=900029a95bd7debb13a5413ec2871202abbad8adee3cad4804f98b9423a50b98
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [3]:
import pandas as pd
import numpy as np
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score,classification_report
import optuna as opt
import torch
import os
import joblib

In [4]:
def make_save_cv_model(i,model_name,model,best_params,optim,output_path="./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/cross_validated_models"):

    ''' This function saves cross validation model in the corresponding directory ( if the path does not exist it creates the path for it'''


    if os.path.exists(os.path.join(output_path,f"{i}_{model_name}_{optim}")):
        joblib.dump(model, os.path.join(output_path,f"{i}_{model_name}_{optim}/{i}_model.z"))
        with open(os.path.join(output_path,f"{i}_{model_name}_{optim}/model_params.txt"),"w+") as file:
            file.write(str(best_params))
    else:
        os.mkdir(os.path.join(output_path,f"{i}_{model_name}_{optim}"))
        joblib.dump(model, os.path.join(output_path,f"{i}_{model_name}_{optim}/{i}_model.z"))
        with open(os.path.join(output_path,f"{i}_{model_name}_{optim}/model_params.txt"),"w+") as file:
            file.write(str(best_params))

In [5]:
def train(fold_dict,fold,model_name,sc_df,tar_col,optim,optim_trial,k_folds=10,tar_cols="",verbose=1):

    ''' this function is used to train the model with parameters optimization using optuna and cross validation using stratified k_folds'''

    y = sc_df[tar_col]
    x = sc_df.drop([tar_col],axis=1)
    model_name = model_name 
    def objective(trial):
      train_index = fold_dict[fold]["train"]
      test_index = fold_dict[fold]["test"]
      clf = TabNetClassifier(n_d=trial.suggest_int("n_d", 8, 64),
                              n_a =trial.suggest_int("n_a", 8, 64),
                              n_steps = trial.suggest_int("n_steps",3,10),
                              gamma =trial.suggest_float("gamma", 1.0, 2.0),
                              n_independent = trial.suggest_int("n_independent",1,5),
                              n_shared = trial.suggest_int("n_shared",1,5),
                              momentum = trial.suggest_float("momentum", 0.01, 0.4),
                              optimizer_fn = torch.optim.Adam,
                              # scheduler_fn = torch.optim.lr_scheduler,
                              # scheduler_params = {"gamma" :trial.suggest_float("sch-gamma", 0.5, 0.95), "step_size": trial.suggest_int("sch_step_size", 10, 20, 2)},
                              verbose = verbose,
                              device_name = "auto"
                              )
      # print(f" train_index :: {train_index}")
      # print(f" test_index :: {test_index}")
      X_train,X_test = x.iloc[train_index,:], x.iloc[test_index,:]
      # print(X_train.shape, X_test.shape)
      X_train, X_test = X_train.to_numpy(dtype=np.float64), X_test.to_numpy(dtype=np.float64)
      Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
      Y_train, Y_test = Y_train.to_numpy(dtype=np.float64), Y_test.to_numpy(dtype=np.float64)
      print(Y_train.shape, Y_test.shape)
      clf.fit(X_train, Y_train,
              eval_set=[(X_test, Y_test)],
              eval_metric=['accuracy'])
      Y_pred = clf.predict(X_test)
      print(classification_report(Y_test, Y_pred, labels=[x for x in range(6)]))
      acc = accuracy_score(Y_pred, Y_test)
      return acc

    print(f"Starting optimization for fold : [{fold}/{k_folds}]")
    study = opt.create_study(direction='maximize')
    study.optimize(objective, n_trials=optim_trial)
    best_params = study.best_params
    print(f" Best params for fold : [{fold}/{k_folds}]")
    print(best_params)
    joblib.dump(best_params,f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/{model_name}/best_params/comp/fold_{fold}_best_params.z")
    with open(f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/{model_name}/best_params/fold_{fold}_best_params.txt", "w+") as file:file.write(str(best_params))
    print(f"Saved best_params at : outputs/{model_name}/best_params/fold_{fold}_best_params.txt")
    train_index = fold_dict[fold]["train"]
    test_index = fold_dict[fold]["test"]
    X_train,X_test = x.iloc[train_index,:], x.iloc[test_index,:]
    # print(X_train.shape, X_test.shape)
    X_train, X_test = X_train.to_numpy(dtype=np.float64), X_test.to_numpy(dtype=np.float64)
    Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
    Y_train, Y_test = Y_train.to_numpy(dtype=np.float64), Y_test.to_numpy(dtype=np.float64)
    clf_model = TabNetClassifier(**study.best_params)
    clf_model.fit(X_train,Y_train)
    Y_pred = clf_model.predict(X_test)
    clf_report = classification_report(Y_test, Y_pred, labels=[x for x in range(6)])
    with open(f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/classification_report/{model_name}_{fold}_classification_report.txt","w+") as file:file.write(str(clf_report))
    accuracy = accuracy_score(Y_pred, Y_test)
    with open(f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/{model_name}/{model_name}_{fold}_accuracy_score.txt","w+") as file:file.write(f" accuracy :: {str(accuracy)}")
    try:
        print("[++] Saving the model and parameters in corresponding directories")
        make_save_cv_model(fold,model_name,clf_model,best_params,optim=optim)
    except:
        print("[-] Failed to save the model")

In [6]:
use_df = pd.read_csv("./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/data/trainable_scaled_balanced.csv")
tar_col = "PCE_categorical"
model_name = "pytorch_tabnet"
optimizer = "Adam"
fold_dict = joblib.load("./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/inputs/fold_vals/fold_data.z")
fold = 0

In [7]:
train(fold_dict = fold_dict,
      fold = fold,
      model_name=model_name,
      sc_df=use_df,
      tar_col=tar_col,
      optim=optimizer,
      optim_trial = 15)
print(f"[++] Ended the training process for fold {fold}")

[I 2022-05-28 07:50:20,252] A new study created in memory with name: no-name-7de4940b-6327-42bf-b2fc-a513419e96b2


Starting optimization for fold : [0/10]
Device used : cuda
(42138,) (4682,)
epoch 0  | loss: 1.2641  | val_0_accuracy: 0.57561 |  0:00:02s
epoch 1  | loss: 0.83787 | val_0_accuracy: 0.64246 |  0:00:05s
epoch 2  | loss: 0.75302 | val_0_accuracy: 0.66489 |  0:00:09s
epoch 3  | loss: 0.717   | val_0_accuracy: 0.67535 |  0:00:12s
epoch 4  | loss: 0.69643 | val_0_accuracy: 0.68432 |  0:00:14s
epoch 5  | loss: 0.68172 | val_0_accuracy: 0.69415 |  0:00:16s
epoch 6  | loss: 0.67226 | val_0_accuracy: 0.70226 |  0:00:18s
epoch 7  | loss: 0.6595  | val_0_accuracy: 0.65271 |  0:00:20s
epoch 8  | loss: 0.65506 | val_0_accuracy: 0.70931 |  0:00:22s
epoch 9  | loss: 0.64652 | val_0_accuracy: 0.70312 |  0:00:24s
epoch 10 | loss: 0.6387  | val_0_accuracy: 0.71871 |  0:00:26s
epoch 11 | loss: 0.63342 | val_0_accuracy: 0.71636 |  0:00:28s
epoch 12 | loss: 0.63176 | val_0_accuracy: 0.71807 |  0:00:30s
epoch 13 | loss: 0.62189 | val_0_accuracy: 0.71188 |  0:00:32s
epoch 14 | loss: 0.62398 | val_0_accuracy:

[I 2022-05-28 07:53:15,846] Trial 0 finished with value: 0.7988039299444681 and parameters: {'n_d': 47, 'n_a': 15, 'n_steps': 3, 'gamma': 1.798843744781617, 'n_independent': 3, 'n_shared': 2, 'momentum': 0.258646784653432}. Best is trial 0 with value: 0.7988039299444681.


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       826
           1       0.69      0.59      0.64       826
           2       0.82      0.96      0.88       825
           3       1.00      1.00      1.00       825
           4       0.53      0.62      0.57       761
           5       0.72      0.55      0.62       619

    accuracy                           0.80      4682
   macro avg       0.79      0.79      0.79      4682
weighted avg       0.80      0.80      0.80      4682

Device used : cuda
(42138,) (4682,)
epoch 0  | loss: 1.50222 | val_0_accuracy: 0.54122 |  0:00:01s
epoch 1  | loss: 1.04124 | val_0_accuracy: 0.54208 |  0:00:04s
epoch 2  | loss: 0.93486 | val_0_accuracy: 0.6038  |  0:00:06s
epoch 3  | loss: 0.87236 | val_0_accuracy: 0.6164  |  0:00:09s
epoch 4  | loss: 0.8372  | val_0_accuracy: 0.63071 |  0:00:11s
epoch 5  | loss: 0.79501 | val_0_accuracy: 0.63413 |  0:00:13s
epoch 6  | loss: 0.78033 | val_0_accuracy: 

[I 2022-05-28 07:56:34,731] Trial 1 finished with value: 0.7926099957283212 and parameters: {'n_d': 13, 'n_a': 63, 'n_steps': 6, 'gamma': 1.9633492677416111, 'n_independent': 1, 'n_shared': 1, 'momentum': 0.2277795110983443}. Best is trial 0 with value: 0.7988039299444681.


...

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       826
           1       0.65      0.61      0.63       826
           2       0.81      0.94      0.87       825
           3       1.00      1.00      1.00       825
           4       0.55      0.62      0.58       761
           5       0.78      0.54      0.64       619

    accuracy                           0.80      4682
   macro avg       0.80      0.79      0.79      4682
weighted avg       0.80      0.80      0.80      4682

Device used : cuda
(42138,) (4682,)
epoch 0  | loss: 2.20465 | val_0_accuracy: 0.54208 |  0:00:06s
epoch 1  | loss: 1.87855 | val_0_accuracy: 0.55318 |  0:00:12s
epoch 2  | loss: 1.28283 | val_0_accuracy: 0.56365 |  0:00:18s
epoch 3  | loss: 1.20222 | val_0_accuracy: 0.57817 |  0:00:24s
epoch 4  | loss: 1.04904 | val_0_accuracy: 0.60273 |  0:00:30s
epoch 5  | loss: 0.92635 | val_0_accuracy: 0.60893 |  0:00:37s
epoch 6  | loss: 0.83149 | val_0_accuracy: 

[I 2022-05-28 09:06:36,058] Trial 13 finished with value: 0.7528833831695857 and parameters: {'n_d': 55, 'n_a': 36, 'n_steps': 10, 'gamma': 1.3036135285393398, 'n_independent': 4, 'n_shared': 2, 'momentum': 0.1556158992696952}. Best is trial 9 with value: 0.8233660828705681.


              precision    recall  f1-score   support

           0       0.99      1.00      0.99       826
           1       0.54      0.57      0.56       826
           2       0.77      0.88      0.82       825
           3       1.00      1.00      1.00       825
           4       0.50      0.51      0.50       761
           5       0.65      0.47      0.55       619

    accuracy                           0.75      4682
   macro avg       0.74      0.74      0.74      4682
weighted avg       0.75      0.75      0.75      4682

Device used : cuda
(42138,) (4682,)
epoch 0  | loss: 1.3161  | val_0_accuracy: 0.54165 |  0:00:04s
epoch 1  | loss: 1.00155 | val_0_accuracy: 0.60252 |  0:00:08s
epoch 2  | loss: 0.97806 | val_0_accuracy: 0.59825 |  0:00:13s
epoch 3  | loss: 0.95033 | val_0_accuracy: 0.61106 |  0:00:17s
epoch 4  | loss: 0.91727 | val_0_accuracy: 0.61448 |  0:00:22s
epoch 5  | loss: 0.87556 | val_0_accuracy: 0.6384  |  0:00:26s
epoch 6  | loss: 0.85868 | val_0_accuracy: 

[I 2022-05-28 09:10:54,094] Trial 14 finished with value: 0.7750961127723195 and parameters: {'n_d': 40, 'n_a': 54, 'n_steps': 5, 'gamma': 1.6089493307373182, 'n_independent': 5, 'n_shared': 3, 'momentum': 0.32911104063460267}. Best is trial 9 with value: 0.8233660828705681.


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       826
           1       0.59      0.64      0.61       826
           2       0.85      0.86      0.85       825
           3       1.00      1.00      1.00       825
           4       0.50      0.56      0.53       761
           5       0.71      0.51      0.60       619

    accuracy                           0.78      4682
   macro avg       0.77      0.76      0.76      4682
weighted avg       0.78      0.78      0.78      4682

 Best params for fold : [0/10]
{'n_d': 55, 'n_a': 45, 'n_steps': 3, 'gamma': 1.6842070860080716, 'n_independent': 5, 'n_shared': 3, 'momentum': 0.3071847290722598}
Saved best_params at : outputs/pytorch_tabnet/best_params/fold_0_best_params.txt
Device used : cuda
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 1.20784 |  0:00:02s
epoch 1  | loss: 0.84975 |  0:00:05s
epoch 2  | loss: 0.76839 |  0:00:08s
epoch 3  

Fold 0 has started running on 20-05-22 


Fold 0 has completed sucessfully on 17:00 20-05-22

Fold 1 has started running at 15:15 21-05-22

Fold 2 has started running at 09:45 22-05-22

Fold 2 has completed sucessfully on 10:58 22-05-22

Fold 3 has started running at 18:40 22-05-22

Fold 3 has completed sucessfully on 22-05-22

Fold 4 completed sucessfully on 21:04 on 22-05-22

Fold 5 started at 18:21 on 23-05-22

Fold 5 completed sucessfully on 19:44 on 23-05-22

Fold 6 started at 12:53 on 24-05-22

Fold 6 has completed at 14:14 on 24-05-22

Fold 7 started at 14:18 on 24-05-22

Fold 7 execution failed due to colab gpu time limit

Fold 7 trial 1 started at 11:00 on 25-05-22

Fold 7 has completed sucessfully at 12:14 on 25-05-22 

Fold 8 has started at 9:38 on 26-05-22

Fold 8 filed due to interrupted internet connection

Fold 8 trial 1 started at 13:38 on 26-05-22

Fold 8 has successfully executed at 15:33 on 26-05-22

Fold 9 has started at 13:35 on 27-05-22

Fold 9 has completed at 14:55 on 27-05-22

Editing with rectified dataset witout duplicacy because of space values

Fold 0 started at 13:21 on 28-05-22

Fold 0 completed sucessfully at 14:46 on 28-05-22